# Download de l'ensemble des bases de données 

## Purpose de ce notebook : 

- Download de l'ensemble des bases de données nécessaire au travail de visualisation dans le cadre de notre projet 
- Cleaning de la base de données afin de pouvoir faire des manipulations dans le cadre de notre projet (drop de colonne, filtrage, etc...)

### Import des bibliothèques nécessaires 

In [1]:
!pip install chardet 
!pip install nbformat
import requests
import zipfile
import os
import pandas as pd
import geopandas as gpd

In [32]:

# Dictionnaire des URLs par année
urls = {
    2022: 'https://www.data.gouv.fr/fr/datasets/r/36314eff-2712-4d6a-b091-7752a5b318bb',
    2021: 'https://www.data.gouv.fr/fr/datasets/r/e6cd3b9f-38e2-47ca-aac6-5acfb9567e01',
    2023: "https://www.data.gouv.fr/fr/datasets/r/e8639487-199c-4e30-8998-1b517148fe29",
}

for year, url_zip in urls.items():
    print(f"\nTraitement de l'année {year}...")

    # Chemin pour stocker temporairement le fichier téléchargé
    file_path = f"data_{year}.zip"

    # Télécharger le fichier
    response = requests.get(url_zip)
    if response.status_code != 200:
        print(f"Erreur lors du téléchargement pour l'année {year}: {response.status_code}")
        continue

    # Enregistrer le fichier localement
    with open(file_path, "wb") as f:
        f.write(response.content)

    # Vérifier si le fichier est un ZIP
    if zipfile.is_zipfile(file_path):
        print(f"Extraction du fichier ZIP pour l'année {year}")
        extract_path = f"data_extracted_{year}"
        if not os.path.exists(extract_path):
            os.makedirs(extract_path)
        with zipfile.ZipFile(file_path, "r") as zip_ref:
            zip_ref.extractall(extract_path)

        # Afficher tous les fichiers extraits pour diagnostic
        print(f"Fichiers extraits pour {year} :")
        extracted_files = []
        for root, _, files in os.walk(extract_path):
            for file in files:
                extracted_files.append(os.path.join(root, file))
                print(f"- {file}")

        # Lire les fichiers extraits si ce sont des CSV
        for file in extracted_files:
            if file.endswith(".csv"):
                gdf = pd.read_csv(file)
                globals()[f"gdf{year}"] = gdf.drop(columns='geometry', errors='ignore')
                print(f"Aperçu des données pour {year} :")
                print(globals()[f"gdf{year}"].head())
    else:
        print(f"Le fichier pour {year} n'est pas un ZIP.")



Traitement de l'année 2022...
Extraction du fichier ZIP pour l'année 2022
Fichiers extraits pour 2022 :
- rpg-bio-2022-national.gpkg
- ._rpg-bio-2022-national.gpkg

Traitement de l'année 2021...
Extraction du fichier ZIP pour l'année 2021
Fichiers extraits pour 2021 :
- rpg-bio-2021-national.gpkg

Traitement de l'année 2023...
Extraction du fichier ZIP pour l'année 2023
Fichiers extraits pour 2023 :
- rpg-bio-2023-national.gpkg
- rpg-bio-2023-national.prj
- rpg-bio-2023-national.cpg
- rpg-bio-2023-national.shx
- rpg-bio-2023-national.dbf
- rpg-bio-2023-national.shp


### Stockage des données dans des variables 

In [33]:
# Chemins vers les fichiers extraits
file_2022 = "data_extracted_2022/rpg-bio-2022-national.gpkg"
file_2021 = "data_extracted_2021/rpg-bio-2021-national.gpkg"
file_2023 = "/home/onyxia/work/AgricultureBio_France/data_extracted_2023/rpg-bio-2023-national/rpg-bio-2023-national.shp" #Choix du .shp en raison d'un manque de data sur le .gpkg

# Charger les fichiers dans des variables distinctes
gdf2022 = gpd.read_file(file_2022)
gdf2021 = gpd.read_file(file_2021)
gdf2023 = gpd.read_file(file_2023)

# Afficher un aperçu des données
print("Aperçu des données pour 2022 :")
print(gdf2022.head())

print("\nAperçu des données pour 2021 :")
print(gdf2021.head())

print("\nAperçu des données pour 2023 :")
print(gdf2023.head())

/opt/conda/lib/python3.12/site-packages/pyogrio/raw.py:198: RuntimeWarning: /home/onyxia/work/AgricultureBio_France/data_extracted_2023/rpg-bio-2023-national/rpg-bio-2023-national.shp contains polygon(s) with rings with invalid winding order. Autocorrecting them, but that shapefile should be corrected using ogr2ogr for example.
  return ogr_read(


Aperçu des données pour 2022 :
       gid  millesime code_culture  \
0  4420687       2022          PPH   
1      333       2022          PTR   
2      337       2022          BTH   
3      464       2022          BTH   
4      480       2022          BTH   

                                         lbl_culture           grp_culture  \
0  Prairie permanente - herbe prédominante (resso...  Prairies permanentes   
1         Autre prairie temporaire de 5 ans ou moins  Prairies temporaires   
2                                 Blé tendre d’hiver            Blé tendre   
3                                 Blé tendre d’hiver            Blé tendre   
4                                 Blé tendre d’hiver            Blé tendre   

   surface_ha code_commune_insee             lbl_commune    code_epci  \
0        0.38              38513                Treffort  200030658.0   
1        2.13              01188                  Illiat  200070118.0   
2        6.10              01136  Cruzilles-lès-Mépi

In [34]:
#Vérification avec le dossier .shp du nombre de data 
len(gdf2023)
gdf2023.columns

Index(['gid', 'millesime', 'code_cultu', 'precision_', 'lbl_cultur',
       'grp_cultur', 'surface_ha', 'code_commu', 'lbl_commun', 'code_epci',
       'code_depar', 'code_regio', 'zone_geo', 'geometry'],
      dtype='object')

In [35]:
# Renommer les colonnes dans gdf2023
gdf2023.rename(columns={'code_depar': 'code_departement', 'code_regio': 'code_region'}, inplace=True)

# Vérifier les nouveaux noms de colonnes
print("Colonnes après renommage :")
print(gdf2023.columns)


Colonnes après renommage :
Index(['gid', 'millesime', 'code_cultu', 'precision_', 'lbl_cultur',
       'grp_cultur', 'surface_ha', 'code_commu', 'lbl_commun', 'code_epci',
       'code_departement', 'code_region', 'zone_geo', 'geometry'],
      dtype='object')


In [36]:
# Liste des codes régionaux pour la métropole
metropolitan_codes = [11, 24, 27, 28, 32, 44, 52, 53, 75, 76, 84, 93, 94]

# Traiter les données pour chaque année
for year in [2021, 2022, 2023]:
    try:
        # Charger le DataFrame de l'année en cours
        gdf = globals().get(f"gdf{year}")
        if gdf is not None:
            # Exclure les valeurs non métropolitaines et NaN
            gdf = gdf.dropna(subset=['code_region'])  # Supprimer les lignes avec code_region NaN
            gdf['code_region'] = gdf['code_region'].astype(int)  # Convertir code_region en int
            
            # Filtrer pour les régions métropolitaines
            gdf_metropole = gdf[gdf['code_region'].isin(metropolitan_codes)]
            
            # Stocker les données dans une variable dynamique
            globals()[f"gdf_metropole{year}"] = gdf_metropole
            
            # Afficher un aperçu des résultats
            print(f"Valeurs uniques de 'code_region' après filtrage pour {year} :")
            print(gdf_metropole['code_region'].unique())
            print(f"Nombre de lignes restantes pour gdf_metropole{year} : {len(gdf_metropole)}\n")
        else:
            print(f"Aucune donnée trouvée pour l'année {year}.")
    except Exception as e:
        print(f"Erreur lors du traitement des données pour {year} : {e}")


/opt/conda/lib/python3.12/site-packages/geopandas/geodataframe.py:1819: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


Valeurs uniques de 'code_region' après filtrage pour 2021 :
[84 44 93 32 76 75 11 27 24 52 53 28 94]
Nombre de lignes restantes pour gdf_metropole2021 : 1025777

Valeurs uniques de 'code_region' après filtrage pour 2022 :
[84 27 75 24 32 44 11 93 76 28 53 52 94]
Nombre de lignes restantes pour gdf_metropole2022 : 1081878

Valeurs uniques de 'code_region' après filtrage pour 2023 :
[76 93 27 75 28 84 24 53 44 11 94 52 32]
Nombre de lignes restantes pour gdf_metropole2023 : 1160919



In [37]:
# Travail sur les données de 'code_region' dans 2021 en raison d'un bug sur la fonction de cleaning 
print("Valeurs uniques de 'code_region' dans gdf2021 :")
print(gdf2021['code_region'].unique())

print(len(gdf2021))

Valeurs uniques de 'code_region' dans gdf2021 :
[84. 44. 93. 32. nan 76. 75. 11. 27. 24. 52. 53. 28. 94.  1.  2.  6.  3.
  4.]
1029200


In [39]:
# Supprimer les valeurs NaN dans code_region
gdf2021 = gdf2021.dropna(subset=['code_region'])

# Convertir code_region en int
gdf2021['code_region'] = gdf2021['code_region'].astype(int)

# Filtrer pour les régions métropolitaines
gdf_metropole2021 = gdf2021[gdf2021['code_region'].isin(metropolitan_codes)]

# Vérifier les valeurs uniques et le nombre de lignes restantes
print("Valeurs uniques de 'code_region' après filtrage :")
print(gdf_metropole2021['code_region'].unique())
print(f"Nombre de lignes restantes : {len(gdf_metropole2021)}")


Valeurs uniques de 'code_region' après filtrage :
[84 44 93 32 76 75 11 27 24 52 53 28 94]
Nombre de lignes restantes : 1025777


### Automatisation du cleaning 

In [40]:
# Dictionnaire des mappings pour les régions
region_mapping = {
    '11': 'Île-de-France',
    '24': 'Centre-Val de Loire',
    '27': 'Bourgogne-Franche-Comté',
    '28': 'Normandie',
    '32': 'Hauts-de-France',
    '44': 'Grand Est',
    '52': 'Pays de la Loire',
    '53': 'Bretagne',
    '75': 'Nouvelle-Aquitaine',
    '76': 'Occitanie',
    '84': 'Auvergne-Rhône-Alpes',
    '93': 'Provence-Alpes-Côte d\'Azur',
    '94': 'Corse'
}

# Fonction de nettoyage pour une base de données donnée
def clean_and_process_data(gdf, year):
    # Étape 1 : Supprimer la colonne geometry si elle existe
    gdf = gdf.drop(columns=['geometry'], errors='ignore')
    
    # Étape 2 : Compter le nombre de parcelles biologiques par département
    parcelles_par_departement = gdf['code_departement'].value_counts().reset_index()
    parcelles_par_departement.columns = ['code_departement', 'nombre_parcelles']
    
    # Étape 3 : Agréger la surface totale des parcelles biologiques par département
    surface_par_departement = gdf.groupby('code_departement')['surface_ha'].sum().reset_index()
    surface_par_departement.columns = ['code_departement', 'surface_totale_ha']
    
    # Fusionner les deux DataFrames
    resultat = parcelles_par_departement.merge(surface_par_departement, on='code_departement')
    resultat = resultat.sort_values(by='surface_totale_ha', ascending=False)
    
    # Étape 4 : Filtrer pour les régions métropolitaines
    gdf = gdf[gdf['code_region'].astype(str).isin(region_mapping.keys())]
    
    # Ajouter une colonne nom_region à partir des codes
    gdf['nom_region'] = gdf['code_region'].astype(str).map(region_mapping)
    
    # Filtrer pour les codes régionaux métropolitains
    gdf_metropole = gdf[gdf['code_region'].isin(metropolitan_codes)]
    
    # Calculer le total des parcelles bio en métropole
    total_metropole = len(gdf_metropole)
    print(f"Le nombre total de parcelles bio en métropole pour {year} est :", total_metropole)
    
    # Étape 5 : Grouper par département pour obtenir le nombre de parcelles par département
    gdf_dep = gdf_metropole.groupby('code_departement').size().reset_index(name='nb_parcelles_bio')
    
    # Étape 6 : Calculer le ratio
    gdf_dep['ratio_parcelles'] = gdf_dep['nb_parcelles_bio'] / total_metropole * 100
    
    # Créer une variable globale pour chaque année
    globals()[f"gdf_dep{year}"] = gdf_dep
    
    # Retourner les résultats et le DataFrame nettoyé
    return gdf, resultat

# Appliquer le nettoyage pour chaque année
for year in urls.keys():
    print(f"\nTraitement des données pour l'année {year}...")
    try:
        # Charger le fichier correspondant à l'année
        gdf = globals().get(f"gdf{year}")
        if gdf is not None:
            # Appliquer la fonction de nettoyage
            gdf_cleaned, resultat = clean_and_process_data(gdf, year)
            
            # Afficher les premiers résultats
            print(f"Aperçu des données regroupées par département pour l'année {year} :")
            print(globals()[f"gdf_dep{year}"].head())
        else:
            print(f"Aucune donnée trouvée pour l'année {year}.")
    except Exception as e:
        print(f"Erreur lors du traitement des données pour {year} : {e}")



Traitement des données pour l'année 2022...
Le nombre total de parcelles bio en métropole pour 2022 est : 1081878
Aperçu des données regroupées par département pour l'année 2022 :
  code_departement  nb_parcelles_bio  ratio_parcelles
0               01              7816         0.722447
1               02              3574         0.330351
2               03              6442         0.595446
3               04             16365         1.512647
4               05             16591         1.533537

Traitement des données pour l'année 2021...
Le nombre total de parcelles bio en métropole pour 2021 est : 1025777
Aperçu des données regroupées par département pour l'année 2021 :
  code_departement  nb_parcelles_bio  ratio_parcelles
0               01              7533         0.734370
1               02              3324         0.324047
2               03              6257         0.609977
3               04             15382         1.499546
4               05             16042        

In [42]:
# Liste des années à traiter
years = [2021, 2022, 2023]

# Traiter les données pour chaque année
for year in years:
    try:
        # Charger les DataFrames correspondants
        gdf = globals().get(f"gdf{year}")
        gdf_metropole = globals().get(f"gdf_metropole{year}")
        
        if gdf is not None and gdf_metropole is not None:
            # Groupement par département
            gdf_dep = gdf_metropole.groupby('code_departement').size().reset_index(name='nb_parcelles_bio')
            
            # Calcul du ratio des parcelles biologiques
            total_parcelles_metropole = len(gdf_metropole)
            gdf_dep['ratio_parcelles'] = gdf_dep['nb_parcelles_bio'] / total_parcelles_metropole * 100
            
            # Fusion avec la géométrie
            gdf_dep = gdf_dep.merge(gdf[['code_departement', 'geometry']], on='code_departement', how='left')
            
            # Convertir en GeoDataFrame
            gdf_dep = gpd.GeoDataFrame(gdf_dep, geometry='geometry')
            
            # Stocker dynamiquement le résultat
            globals()[f"gdf_dep{year}"] = gdf_dep
            
            # Afficher un aperçu des résultats
            print(f"Traitement terminé pour gdf_dep{year}:")
            print(gdf_dep.head())
        else:
            print(f"Données manquantes pour l'année {year}.")
    except Exception as e:
        print(f"Erreur lors du traitement des données pour {year}: {e}")



Traitement terminé pour gdf_dep2021:
  code_departement  nb_parcelles_bio  ratio_parcelles  \
0               01              7533          0.73437   
1               01              7533          0.73437   
2               01              7533          0.73437   
3               01              7533          0.73437   
4               01              7533          0.73437   

                                            geometry  
0  MULTIPOLYGON (((5.03465 45.82035, 5.03338 45.8...  
1  MULTIPOLYGON (((5.70101 46.04451, 5.69986 46.0...  
2  MULTIPOLYGON (((5.71121 46.03814, 5.71121 46.0...  
3  MULTIPOLYGON (((5.70964 46.03805, 5.70964 46.0...  
4  MULTIPOLYGON (((5.66236 46.14657, 5.66261 46.1...  
Traitement terminé pour gdf_dep2022:
  code_departement  nb_parcelles_bio  ratio_parcelles  \
0               01              7816         0.722447   
1               01              7816         0.722447   
2               01              7816         0.722447   
3               01       

,code_departement,nb_parcelles_bio,ratio_parcelles,geometry
322087,26,39038,3.608355,"MULTIPOLYGON (((4.88821 44.26197, 4.88756 44.2..."
283068,26,39038,3.608355,"MULTIPOLYGON (((5.44353 44.37956, 5.44339 44.3..."
283069,26,39038,3.608355,"MULTIPOLYGON (((5.63816 44.6455, 5.63809 44.64..."
283058,26,39038,3.608355,"MULTIPOLYGON (((5.536 44.13147, 5.5354 44.1325..."
283059,26,39038,3.608355,"MULTIPOLYGON (((5.57435 44.15253, 5.57439 44.1..."
...,...,...,...,...
1081575,94,24,0.002218,"MULTIPOLYGON (((2.56821 48.70704, 2.56825 48.7..."
1081576,94,24,0.002218,"MULTIPOLYGON (((2.56805 48.70581, 2.56826 48.7..."
1081577,94,24,0.002218,"MULTIPOLYGON (((2.56915 48.70614, 2.5693 48.70..."
1081578,94,24,0.002218,"MULTIPOLYGON (((2.57545 48.80099, 2.57575 48.8..."
